In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as  plt
import seaborn as sns
import string
import keras
import os
import cv2 
import pickle

from nltk.tokenize import word_tokenize , RegexpTokenizer
from nltk.corpus import stopwords
from nltk import FreqDist


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Model
from keras.layers.merge import concatenate
from keras.layers.pooling import GlobalMaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.models import InputLayer , Sequential
from keras.layers import Conv2D ,Dense, ReLU, Flatten ,LSTM ,TimeDistributed ,Input ,TimeDistributed ,Embedding ,RepeatVector

%matplotlib inline

Using TensorFlow backend.


In [2]:
image_dir = "C:\\Users\\vinayak tyagi\\Desktop\\images"
raw_txt = "C:\\Users\\vinayak tyagi\\Desktop\\txt_data.txt"

In [3]:
# reading the text file 
def load_txt_file(filename):
    file = open(filename , 'r')
    txt = file.read()
    file.close()
    return txt

In [4]:
data = load_txt_file(raw_txt)
print(data)

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin .
1001773457_577c3a7d70.jpg#0	A black dog and a spotted dog are fighting
1001773457_577c3a7d70.jpg#1	A black dog and a tri-colored dog playing with each other on the road .
1001773457_577c3a7d70.jpg#2	A black dog and a white dog with brown spots are staring at each other in the street .
1001773457_577c3a7d70.jpg#3	Two dogs of different breeds looking at each other on the road .
1001773457_577c3a7d70.jpg#4	Two dogs on pavement moving toward each other .
1002674143_1b742ab4b8.jpg#0	A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .
1002674143_1b

In [5]:
# now cleaning the data 
def cleaning(data):
    word_lst = []; img_id_lst = [] 
    for line in data.split("\n"):
        line = line.lower()
        tokens = word_tokenize(line)
        if tokens[2] == "0":
            img_id =  tokens[0].split(".")[0]
            lst = [i for i in tokens[3:] if i.isalpha()]
            word_lst.append(lst)
            img_id_lst.append(img_id)
    return word_lst, img_id_lst

In [6]:
desc , img_id = cleaning(data)

In [7]:
# now loading the images for feature
def making_img_data(desc, img_id_lst):
    image_lst = []; desc_lst = []
    in_layer = Input(shape=(224, 224, 3))
    model = VGG16(include_top=False, input_tensor=in_layer)
    for i in img_id_lst:
        img_uri = image_dir+ "\\" + i + ".jpg"
        image = load_img(img_uri, target_size = (224, 224))
        image = img_to_array(image)

        desc_lst.append(desc[img_id_lst.index(i)])
#         reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
#         prepare the image for the VGG model
        image = preprocess_input(image)
#         get features
        feature = model.predict(image, verbose=0)
        image_lst.append(feature)
     
    print(image_lst[0])
    return image_lst , desc_lst

In [ ]:
img_feature , desc = making_img_data(desc, img_id)

In [ ]:
train_x , test_x, train_y , test_y = train_test_split(img_feature , desc , test_size = 0.2 , shuffle = False)

In [ ]:
# def my_funtion(data_tokenizer):
#     lst = []
#     for i in data_tokenizer:
# #         print(i)
#         sent = " ".join(i)
#         lst.append(sent)
# #     print(lst)
#     return lst

In [ ]:
# making the untokenize word's for max len
def untokenize(tokenize_data):
    lst = []; word_lst = []; words_len_lst = []
    for i in tokenize_data:
#         print(len(i))
        words_len_lst.append(len(i))
        for j in i:
            word_lst.append(j)
            lst.append(len(j))
#     print(word_lst)
    print("max len--> ",max(lst))
    return word_lst, words_len_lst,  max(lst)
                    
# now returning the voucab size
def vocab_size(tokenize_data):
    lst = []
    for i in tokenize_data:
#         lst = [lst.append(p) for p in i if p not in lst]
         for p in i:
            if p not in lst:
                 lst.append(p)
#     print("vocab--> ",lst)
    print("vocab--> ",len(lst))
    return len(lst)  

In [ ]:
# max_len = untokenize(train_y)
vocab_size = vocab_size(train_y)
word_lst , words_len_lst, max_len  = untokenize(train_y)

In [ ]:
def define_model(vocab_size, max_length):
	# feature extractor (encoder)
	inputs1 = Input(shape=(7, 7, 512))
	fe1 = GlobalMaxPooling2D()(inputs1)
	fe2 = Dense(128, activation='relu')(fe1)
	fe3 = RepeatVector(max_length)(fe2)
	# embedding
	inputs2 = Input(shape=(max_length,))
	emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
	emb3 = LSTM(256, return_sequences=True)(emb2)
	emb4 = TimeDistributed(Dense(128, activation='relu'))(emb3)
	# merge inputs
	merged = concatenate([fe3, emb4])
	# language model (decoder)
	lm2 = LSTM(500)(merged)
	lm3 = Dense(500, activation='relu')(lm2)
	outputs = Dense(vocab_size, activation='softmax')(lm3)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	print(model.summary())
##	plot_model(model, show_shapes=True, to_file='plot.png')
	return model


In [ ]:
model = define_model(vocab_size , max_len)

In [ ]:
len(words_len_lst)

In [ ]:
# train_y

In [ ]:
n_batches_per_epoch = int(len(train_x) / 2)
model.fit([np.array(train_x).reshape(-1,7,7,512), np.array(train_y).reshape(-1,13)] , steps_per_epoch=n_batches_per_epoch, epochs=50, verbose=2)

In [ ]:
# y = my_funtion(train_y)
# print(len(y))
# desc_data = np.array(y).reshape(-1,1)
# print(desc_data)
# print(len(desc_data))

In [ ]:
# print(train_x[0])

In [ ]:
# train_data = np.array(train_y).reshape()
# m = int(len(train_y)/2)
# model.fit([np.array(train_x), desc_data], epochs=50 , verbose = 2)